#### Импорты

In [ ]:
import requests
import pandas as pd
import time
import random
import json

#### Заголовки для браузерного поведения

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "Origin": "https://www.wildberries.ru",
    "Referer": "https://www.wildberries.ru/"
}

#### Получение товара (imtId) по артикулу (SKU)

In [ ]:
def get_item_root_id(sku):
    url = f"https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1257786&spp=30&nm={sku}"
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f"Ошибка API карточки товара: {response.status_code}")
            return None

        data = response.json()
        products = data.get("data", {}).get("products", [])
        if not products:
            print(f"SKU {sku} не найден или нет в наличии.")
            return None

        root_id = products[0].get("root")
        print(f"SKU: {sku} -> RootID: {root_id}")
        return root_id
    except Exception as e:
        print(f"Ошибка при получении rootId для {sku}: {e}")
        return None

#### Скачивание отзывов с разных серверов WB

In [ ]:
def get_reviews(root_id):
    hosts = [
        "https://feedbacks1.wb.ru",
        "https://feedbacks2.wb.ru"
    ]

    for host in hosts:
        url = f"{host}/feedbacks/v1/{root_id}"
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)

            if response.status_code == 200:
                data = response.json()
                feedbacks = data.get("feedbacks", [])

                # Если отзывы найдены — возвращаем их
                if feedbacks:
                    print(f"Успешно: найдено {len(feedbacks)} отзывов на {host}")
                    return feedbacks
                # Если вернулся пустой список, возможно, отзывы на другом сервере -> continue

        except Exception as e:
            print(f"Ошибка запроса к {host}: {e}")
            continue

    print("Отзывы не найдены ни на одном сервере (или их нет у товара).")
    return []

#### Парсинг и сохранение в CSV

In [ ]:
def parse_wb_reviews(sku_list, output_file="reviews.csv", min_length=15):
    all_reviews = []

    for sku in sku_list:
        print(f"\n--- Обработка товара: {sku} ---")
        root_id = get_item_root_id(sku)

        if not root_id:
            continue

        raw_reviews = get_reviews(root_id)

        # Парсинг JSON в удобный формат
        count_for_sku = 0
        for r in raw_reviews:
            text = r.get("text", "")

            # Фильтр коротких/пустых отзывов
            if not text or len(text) < min_length:
                continue

            # Сборка полного текста
            full_text = text
            pros = r.get("pros", "")
            cons = r.get("cons", "")
            if pros: full_text += f" Плюсы: {pros}"
            if cons: full_text += f" Минусы: {cons}"

            review_data = {
                "sku": sku,
                "rating": r.get("productValuation"),
                "text": full_text.strip(),
                "date": r.get("createdDate")
            }
            all_reviews.append(review_data)
            count_for_sku += 1

        print(f"Сохранено {count_for_sku} полезных отзывов для {sku}")
        time.sleep(random.uniform(1.5, 3.0))

    # Сохраняем в CSV
    if all_reviews:
        df = pd.DataFrame(all_reviews)
        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\nГотово! Всего собрано отзывов: {len(df)}. Файл: {output_file}")
        return df
    else:
        print("\nНе удалось собрать ни одного отзыва.")
        return None

In [3]:
if __name__ == "__main__":
    target_skus = [
       "371713881",
       "297946589",
       "355795507",
       "313224783",
       "371712685",
       "143785198",
       "583470697",
       "583470698",

       "270762735",
       "414374585",
       "388037375",
       "196048385",
       "274174984",
       "386101148",
       "188084589",
       "258555240",
       "210869633",
       "210968493",
       "224511097",

       "497900366",
       "613479599",
       "442134036",
       "505156251",
       "504055842",
       "614779017",
       "15484560",
       "582100333",
       "254602900",

       "178163370",
       "385644989",
       "142729489",
       "415537538",
       "220118174",
       "197868953",
       "254910113",

       "173274282",
       "180272902",
       "181336980",
       "206008411",
       "179019554",
       "57675502",
       "190259584",
       "167121582",
       "156976606",
       "243772599",
       "243772597",
       "168701215",
       "10405454",
       "397623578", # пульт
       "397623578",
       "668778603",
       "197745095",
       "150303509",
       "251351415",

       "495894122",
       "173729620",
       "224101962",
       "252631084",
       "179228616",
       "14775947",
       "158465356",
       "158465217",
       "256828630",
       "504902368",
       "516960378",

       "535327046",
       "262371180",
       "387066874",

       "351028376",
       "186700783",
       "143414361",
       "227649169",
       "182168410",
       "141352845",

       "44516181",
       "313139869",
       "173628800",
       "498185137",

       "220094688",
       "148637843",
       "293967354",
       "373143395",
       "152113571",
       "303914030",
       "11452042",
       "294206895",
       "255039100",

       "371163380",
       "368655900",
       "431811813",
       "435014975",

       "296709004",
       "152985156",
       "229223736",
       "399525786",

       "211786166",
       "197401135",
       "197401134",
       "240450670",
       "317879610",
       "208174079",
    ]

    parse_wb_reviews(target_skus)


--- Обработка товара: 371713881 ---
SKU: 371713881 -> RootID: 363330079
Успешно: найдено 1000 отзывов на https://feedbacks2.wb.ru
Сохранено 226 полезных отзывов для 371713881

--- Обработка товара: 297946589 ---
SKU: 297946589 -> RootID: 280568696
Успешно: найдено 927 отзывов на https://feedbacks2.wb.ru
Сохранено 300 полезных отзывов для 297946589

--- Обработка товара: 355795507 ---
SKU: 355795507 -> RootID: 342696880
Успешно: найдено 1000 отзывов на https://feedbacks2.wb.ru
Сохранено 199 полезных отзывов для 355795507

--- Обработка товара: 313224783 ---
SKU: 313224783 -> RootID: 294422394
Успешно: найдено 1000 отзывов на https://feedbacks2.wb.ru
Сохранено 183 полезных отзывов для 313224783

--- Обработка товара: 371712685 ---
SKU: 371712685 -> RootID: 363330079
Успешно: найдено 1000 отзывов на https://feedbacks2.wb.ru
Сохранено 226 полезных отзывов для 371712685

--- Обработка товара: 143785198 ---
SKU: 143785198 -> RootID: 122293388
Успешно: найдено 535 отзывов на https://feedback